In [1]:
#Before running, make sure you have the shapefiles Bangladesh_Zila.shx and Bangladesh_Zila.shp in the same folder as this script!
#Pip/Conda install relevant libraries before running!
#Import needed libraries
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
from folium import plugins
from folium.plugins import Fullscreen, PolyLineTextPath
import branca.colormap as cm
import vincent, json
from colour import Color
from datetime import datetime
import time
from selenium import webdriver
from random import randint
from random import shuffle
import os
import imageio
import glob
import pathlib
import mysql.connector
import sys
startTime = datetime.now()
starttime = time.time()
timeout = time.time() + 60*1   # 10 minutes from now

In [2]:
roadsOrigin = '_roads3.csv'
bridgesOrigin = 'BMMS_overview.xlsx'
segmentOrigin = 'SimioSpreadsheetAss4_part2.xlsx'
# Import the data files
segments = pd.read_excel(segmentOrigin)
roads3 = pd.read_csv(roadsOrigin)
bridges = pd.read_excel(bridgesOrigin)
RoadName = 'N1'
#Change these locations before running the file, however, this entry needs to be after the input because the variable 'RoadName' is used as an input to obtain the data
location = 'RMMS/'+RoadName+'.traffic.htm'
wlocation = 'RMMS/'+RoadName+'.widths.processed.txt'
ss = segments['Object Name'].str.split('_', expand=True).astype(str)[0:-1]
ss.columns = ['Road', 'LRP Start']
ss.sort_values(by = 'LRP Start', inplace = True, ascending = False)
ss.reset_index(inplace = True, drop = True)

In [3]:
# importing data from sql
cnx = mysql.connector.connect(user='epa1351user',
                              password='xgt65RR##',
                              database='epa1351group14')
cursor = cnx.cursor()
df = pd.read_sql('SELECT * FROM roads', con=cnx)
df
cnx.close()

In [4]:
# first making a list of all lrp's that contain a bridge
bridgelist = []
bridgesN1 = bridges[bridges.road == 'N1']
LRPbridges = list(bridgesN1.LRPName)
for i in LRPbridges:
    up = str.upper(i)
    bridgelist.append(up)
LRPbridges = LRPbridges[0:162]

In [5]:
z = 0
def makemap1():
    global ss
    global roads3
    global bridges
    global z
    print("New update identified! \nThank you for your patience. Upon completion, the image will be loaded automatically. \nAfter completion, you can open file: Map_N1_"+str(z)+".html in the HTML_Files folder to inspect the update in further detail.")
    cnx = mysql.connector.connect(user='epa1351user',
                              password='xgt65RR##',
                              database='epa1351group14')
    cursor = cnx.cursor()
    data = pd.read_sql('SELECT * FROM roads', con=cnx)
    cnx.close()
    data = data[1:-1]

    # importing data from sql
    cnx = mysql.connector.connect(user='epa1351user',
                                  password='xgt65RR##',
                                  database='epa1351group14')
    cursor = cnx.cursor()
    tf = pd.read_sql('SELECT * FROM traveltimes', con=cnx)
    cnx.close()

    normaltimes = [0.248924, 0.330286, 0.255289, 0.344205, 0.412210, 0.166035, 0.124393, 0.085473,
                  0.148808, 0.304980, 0.29109, 0.290473, 0.076435, 0.207639, 0.357661, 0.308615, 
                  0.183085, 0.015438, 0.038084, 0.376641, 0.240937, 0.017052]

    lasttime = np.array(tf.iloc[-1, 1:].values)
    lasttime = lasttime.astype(float)
    deltat = np.array(lasttime) - normaltimes

    #N1 is all the roads that are in N1
    N1 = roads3[roads3.road == 'N1']
    #Change the bridges file so that it has the same 'properties' as the N1 dataframe
    roadbridges = bridges.loc[:,('road','chainage','LRPName','lat', 'lon', 'type', 'name', 'condition', 'length')]
    roadbridges.columns = ['road','chainage','lrp','lat', 'lon', 'type', 'name', 'condition', 'length']
    #Set the type of the bridges as 'bridge'
    roadbridges.type = 'Bridge'
    #Append both dataframes
    N1 = N1.append(roadbridges[roadbridges.road == 'N1'])

    lrps = data['RoadName'].str.split('_', expand=True).astype(str)
    lrps.columns = ['ID', 'LRP Start', 'LRP End']
    dt = pd.concat([data, lrps], axis = 1)
    dt = dt.drop(['ID'], axis = 1)
    dt.reset_index(drop = True, inplace = True)
    x0 = dt[0:dt[dt['LRP Start'] == ss['LRP Start'][0]].index.values[0]].iloc[:,1:]
    c0 = x0.AmountCars.sum()
    t0 = x0.AmountTrucks.sum()
    m0 = x0.AmountBusses.sum()
    lrpStart0 = x0['LRP Start'].iloc[0]
    lrpEnd0 = x0['LRP End'].iloc[-1]
    df1 = pd.DataFrame(data = [[c0, t0, m0, lrpStart0, lrpEnd0]], columns = dt.columns[1:])

    for i in range(1, len(ss)):
        xi = dt[dt[dt['LRP Start'] == ss['LRP Start'][i-1]].index.values[0]:dt[dt['LRP Start'] == ss['LRP Start'][i]].index.values[0]].iloc[:,1:]
        xi.reset_index(inplace = True, drop = True)
        ci = xi.AmountCars.sum()
        ti = xi.AmountTrucks.sum()
        mi = xi.AmountBusses.sum()
        lrpStarti = xi['LRP Start'].iloc[0]
        lrpEndi = xi['LRP End'].iloc[-1]
        nextrowi = pd.DataFrame([[ci, ti, mi, lrpStarti, lrpEndi]], columns = df1.columns)
        df1 = df1.append(nextrowi)

    xf = dt[dt[dt['LRP Start'] == ss['LRP Start'].iloc[-1]].index.values[0]:].iloc[:,1:]
    cf = xf.AmountCars.sum()
    tf = xf.AmountTrucks.sum()
    mf = xf.AmountBusses.sum()
    lrpStartf = xf['LRP Start'].iloc[0]
    lrpEndf = xf['LRP End'].iloc[-1]
    nextrowf = pd.DataFrame([[cf, tf, mf, lrpStartf, lrpEndf]], columns = df1.columns)
    df1 = df1.append(nextrowf)
    df1.reset_index(inplace = True, drop = True)

    df1['Lat Start'] = np.nan
    df1['Lat End'] = np.nan
    df1['Lon Start'] = np.nan
    df1['Lon End'] = np.nan

    for i in range(len(df1)):
        x = N1[N1.lrp == df1['LRP Start'][i]]
        y = N1[N1.lrp == df1['LRP End'][i]]
        (df1['Lat Start'][i], df1['Lon Start'][i], df1['Lat End'][i], df1['Lon End'][i]) = (x.lat.iloc[0], x.lon.iloc[0], y.lat.iloc[0], y.lon.iloc[0])
    df = df1.copy()
    df['SumTraffic'] = np.nan
    df['color'] = np.nan

    for i in range(len(df)):
        df.SumTraffic[i] = df.AmountCars[i] + df.AmountTrucks[i] + df.AmountBusses[i]
    df.color = df.SumTraffic - df.SumTraffic.mean() / df.SumTraffic.std()
    labels = df.copy()
    LatsLonsR = labels[['Lat Start', 'Lon Start']]

    df['DeltaTime'] = deltat
    df['DeltaTimeColor'] = np.nan
    for i in range(len(df)):
        if df.DeltaTime[i] <= 0.01:
            df['DeltaTimeColor'][i] = 'green'
        elif df.DeltaTime[i] <= 0.5:
            df['DeltaTimeColor'][i] = 'yellow'
        elif df.DeltaTime[i] <= 1:
            df['DeltaTimeColor'][i] = 'orange'
        elif df.DeltaTime[i] >= 1:
            df['DeltaTimeColor'][i] = 'red'    
        else:
            df['DeltaTimeColor'][i] = 'black'

    #Focus the zoom on Bangladesh
    lat0=23
    lon0=91
    zs = 9

    #Set the values from the dataframes into lists
    locationlistR = LatsLonsR.values.tolist()
    roadlist = df[['Lat Start', 'Lon Start']].values.tolist()
    endloc = [df.iloc[-1]['Lat End'], df.iloc[-1]['Lon End']]
    roadlist.append(endloc)

    #Read the shapefiles
    lotes = gpd.read_file('Bangladesh_Zilas.shp')
    lotes.crs = {'init' : 'epsg:3857'}
    geoPath = lotes.geometry.to_json()

    #The original map (mapb) is therefore a Folium Map
    mapb = folium.Map([lat0, lon0], zoom_start=zs, tiles='openstreetmap',API_key='wrobstory.map-12345678')

    #Create featuregroups (these are individual layers that can be turned off and on in the top right corner of the map)
    vg = folium.FeatureGroup(name = 'Delay Time')
    pg = folium.FeatureGroup(name = 'Province Borders')

    #Add a layer of the province borders
    folium.GeoJson(lotes).add_to(pg)

    for i in range(len(roadlist)-1):
        #Create chartdata (a dataframe of the counts)
        traffic = [df.AmountCars[i], df.AmountTrucks[i], df.AmountBusses[i]]
        chartdata = pd.DataFrame(traffic, index=['Cars', 'Trucks', 'Busses'])
        #Plot the data using 'vincent' so that the plot can appear in a popup
        chart = vincent.Bar(chartdata, width = 200, height = 300)
        trafficpopup = folium.Popup()
        chart.axis_titles(x=(df['LRP Start'][i] + '-' + df['LRP End'][i] + ": " + str("{0:.0f}".format(df.SumTraffic[i])) + " vehicles"), y='')
        folium.Vega(chart, height=350, width=330).add_to(trafficpopup)
        #Create a popup and add it to the traffic marker cluster
        my_PolyLine_Vul=folium.PolyLine(locations=[roadlist[i], roadlist[i+1]],weight= 8, color = df.DeltaTimeColor[i], popup = trafficpopup)
        vg.add_child(my_PolyLine_Vul)
    
    if 'bblist' in globals():
        for i in range(len(bblist)):
            breakbridges = bridges[(bridges.LRPName == bblist[i]) & (bridges.road == 'N1')]
            breakbridges.reset_index(inplace = True, drop = True)
            poplist = [breakbridges.lat[0], breakbridges.lon[0]]
            bridgeinfo = bridgeinfo = '<b>This bridge seems permanently broken!</b><br>Bridge: ' + breakbridges.LRPName[0] + '<br>Type: '  + breakbridges.type[0]
            folium.Marker(poplist, popup = bridgeinfo, icon=folium.Icon(prefix = 'fa', icon = 'road', color='black')).add_to(vg)

    if RndBridge == 1:
        breakbridges = bridges[(bridges.LRPName == LRPbridges[z]) & (bridges.road == 'N1')]
        breakbridges.reset_index(inplace = True, drop = True)
        poplist = [breakbridges.lat[0], breakbridges.lon[0]]
        bridgeinfo = bridgeinfo = '<b>This bridge is broken!</b><br>Bridge: ' + breakbridges.LRPName[0] + '<br>Type: '  + breakbridges.type[0]
        folium.Marker(poplist, popup = bridgeinfo, icon=folium.Icon(prefix = 'fa', icon = 'road', color='red')).add_to(vg)
        if z >= 1:
            breakbridges = bridges[(bridges.LRPName == LRPbridges[z-1]) & (bridges.road == 'N1')]
            breakbridges.reset_index(inplace = True, drop = True)
            poplist = [breakbridges.lat[0], breakbridges.lon[0]]
            bridgeinfo = bridgeinfo = '<b>This bridge was broken, but it looks like the problem is fixed now!</b><br>Bridge: ' + breakbridges.LRPName[0] + '<br>Type: '  + breakbridges.type[0]
            folium.Marker(poplist, popup = bridgeinfo, icon=folium.Icon(prefix = 'fa', icon = 'road', color='blue')).add_to(vg)
    

    legend_html =   '''
                <div style="position: fixed; 
                            bottom: 10px; left: 10px; width: 300px; height: 320px; background-color: white;
                            border:2px solid grey; z-index:9999; font-size:14px;
                            ">&nbsp; <i> Last update at ''' + str((z + 1) % 24) + ''':00 </i> <br> <br>
                              &nbsp; <b> <u> Legend </u> </b> <br>
                              &nbsp; <i class="fa fa-map-marker fa-2x" style="color:black"></i> - Bridge is permanently broken <br>
                              &nbsp; <i class="fa fa-map-marker fa-2x" style="color:red"></i> - Bridge just broke down <br>
                              &nbsp; <i class="fa fa-map-marker fa-2x" style="color:blue"></i> - Bridge has just been repaired <br> <br>
                              &nbsp; <i class="fa fa-minus fa-2x" style="color:black"></i> - No traffic information yet <br>
                              &nbsp; <i class="fa fa-minus fa-2x" style="color:green"></i> - No significant delays <br>
                              &nbsp; <i class="fa fa-minus fa-2x" style="color:yellow"></i> -  Less than 30 minutes of delay<br>
                              &nbsp; <i class="fa fa-minus fa-2x" style="color:orange"></i> - Less than 1 hour of delay <br>
                              &nbsp; <i class="fa fa-minus fa-2x" style="color:red"></i> -  Over 1 hour of delay <br>
                </div>
                '''

    mapb.get_root().html.add_child(folium.Element(legend_html))
    
    #Add all the layers to the map
    #Allow fullscreen
    mapb.add_child(pg)
    mapb.add_child(vg)
    mapb.add_child(folium.LayerControl(collapsed = False))
    mapb.add_child(folium.plugins.Fullscreen())

    pathlib.Path('PNG_Files').mkdir(parents=True, exist_ok=True)
    pathlib.Path('HTML_Files').mkdir(parents=True, exist_ok=True) 

    mapb.save(outfile = 'HTML_Files/Map_'+RoadName+'_'+str(z)+'.html')
    mapb.save(outfile = 'Map_'+RoadName+'.html')
    #Link to chromedriver
    z = z + 1

    
    
    
    
def makemap2():
    global ss
    global roads3
    global bridges
    global z
    print("New update identified! \nThank you for your patience. Upon completion, the image will be loaded automatically. \nAfter completion, you can open file: Map_N1_"+str(z)+".html in the HTML_Files folder to inspect the update in further detail.")
    cnx = mysql.connector.connect(user='epa1351user',
                              password='xgt65RR##',
                              database='epa1351group14')
    cursor = cnx.cursor()
    data = pd.read_sql('SELECT * FROM roads', con=cnx)
    cnx.close()
    data = data[1:-1]

    # importing data from sql
    cnx = mysql.connector.connect(user='epa1351user',
                                  password='xgt65RR##',
                                  database='epa1351group14')
    cursor = cnx.cursor()
    tf = pd.read_sql('SELECT * FROM traveltimes', con=cnx)
    cnx.close()

    normaltimes = [0.248924, 0.330286, 0.255289, 0.344205, 0.412210, 0.166035, 0.124393, 0.085473,
                  0.148808, 0.304980, 0.29109, 0.290473, 0.076435, 0.207639, 0.357661, 0.308615, 
                  0.183085, 0.015438, 0.038084, 0.376641, 0.240937, 0.017052]

    lasttime = np.array(tf.iloc[-1, 1:].values)
    lasttime = lasttime.astype(float)
    deltat = np.array(lasttime) - normaltimes

    #N1 is all the roads that are in N1
    N1 = roads3[roads3.road == 'N1']
    #Change the bridges file so that it has the same 'properties' as the N1 dataframe
    roadbridges = bridges.loc[:,('road','chainage','LRPName','lat', 'lon', 'type', 'name', 'condition', 'length')]
    roadbridges.columns = ['road','chainage','lrp','lat', 'lon', 'type', 'name', 'condition', 'length']
    #Set the type of the bridges as 'bridge'
    roadbridges.type = 'Bridge'
    #Append both dataframes
    N1 = N1.append(roadbridges[roadbridges.road == 'N1'])

    lrps = data['RoadName'].str.split('_', expand=True).astype(str)
    lrps.columns = ['ID', 'LRP Start', 'LRP End']
    dt = pd.concat([data, lrps], axis = 1)
    dt = dt.drop(['ID'], axis = 1)
    dt.reset_index(drop = True, inplace = True)
    x0 = dt[0:dt[dt['LRP Start'] == ss['LRP Start'][0]].index.values[0]].iloc[:,1:]
    c0 = x0.AmountCars.sum()
    t0 = x0.AmountTrucks.sum()
    m0 = x0.AmountBusses.sum()
    lrpStart0 = x0['LRP Start'].iloc[0]
    lrpEnd0 = x0['LRP End'].iloc[-1]
    df1 = pd.DataFrame(data = [[c0, t0, m0, lrpStart0, lrpEnd0]], columns = dt.columns[1:])

    for i in range(1, len(ss)):
        xi = dt[dt[dt['LRP Start'] == ss['LRP Start'][i-1]].index.values[0]:dt[dt['LRP Start'] == ss['LRP Start'][i]].index.values[0]].iloc[:,1:]
        xi.reset_index(inplace = True, drop = True)
        ci = xi.AmountCars.sum()
        ti = xi.AmountTrucks.sum()
        mi = xi.AmountBusses.sum()
        lrpStarti = xi['LRP Start'].iloc[0]
        lrpEndi = xi['LRP End'].iloc[-1]
        nextrowi = pd.DataFrame([[ci, ti, mi, lrpStarti, lrpEndi]], columns = df1.columns)
        df1 = df1.append(nextrowi)

    xf = dt[dt[dt['LRP Start'] == ss['LRP Start'].iloc[-1]].index.values[0]:].iloc[:,1:]
    cf = xf.AmountCars.sum()
    tf = xf.AmountTrucks.sum()
    mf = xf.AmountBusses.sum()
    lrpStartf = xf['LRP Start'].iloc[0]
    lrpEndf = xf['LRP End'].iloc[-1]
    nextrowf = pd.DataFrame([[cf, tf, mf, lrpStartf, lrpEndf]], columns = df1.columns)
    df1 = df1.append(nextrowf)
    df1.reset_index(inplace = True, drop = True)

    df1['Lat Start'] = np.nan
    df1['Lat End'] = np.nan
    df1['Lon Start'] = np.nan
    df1['Lon End'] = np.nan

    for i in range(len(df1)):
        x = N1[N1.lrp == df1['LRP Start'][i]]
        y = N1[N1.lrp == df1['LRP End'][i]]
        (df1['Lat Start'][i], df1['Lon Start'][i], df1['Lat End'][i], df1['Lon End'][i]) = (x.lat.iloc[0], x.lon.iloc[0], y.lat.iloc[0], y.lon.iloc[0])
    df = df1.copy()
    df['SumTraffic'] = np.nan
    df['color'] = np.nan

    for i in range(len(df)):
        df.SumTraffic[i] = df.AmountCars[i] + df.AmountTrucks[i] + df.AmountBusses[i]
    df.color = df.SumTraffic - df.SumTraffic.mean() / df.SumTraffic.std()
    #Create a colour scale ranging from red to green
    red = Color("red")
    colors = list(red.range_to(Color("green"),10))
    #put the values in a list so they can later be read
    l = []
    for i in range(len(colors)):
        l.append("%s" % colors[i].hex)
    df['ColorCode'] = np.nan
    for i in range(len(df)):
        x = int(((df['color'][i] - df['color'].max()) / (df['color'].min() - df['color'].max()) * 10) - 1)
        df.ColorCode[i] = l[x]
    labels = df.copy()
    LatsLonsR = labels[['Lat Start', 'Lon Start']]

    df['DeltaTime'] = deltat

    #Focus the zoom on Bangladesh
    lat0=23
    lon0=91
    zs = 9

    #Set the values from the dataframes into lists
    locationlistR = LatsLonsR.values.tolist()
    roadlist = df[['Lat Start', 'Lon Start']].values.tolist()
    endloc = [df.iloc[-1]['Lat End'], df.iloc[-1]['Lon End']]
    roadlist.append(endloc)

    #Read the shapefiles
    lotes = gpd.read_file('Bangladesh_Zilas.shp')
    lotes.crs = {'init' : 'epsg:3857'}
    geoPath = lotes.geometry.to_json()

    #The original map (mapb) is therefore a Folium Map
    mapb = folium.Map([lat0, lon0], zoom_start=zs, tiles='openstreetmap',API_key='wrobstory.map-12345678')

    #Create featuregroups (these are individual layers that can be turned off and on in the top right corner of the map)
    vg = folium.FeatureGroup(name = 'Traffic Density')
    pg = folium.FeatureGroup(name = 'Province Borders')

    #Add a layer of the province borders
    folium.GeoJson(lotes).add_to(pg)
    
    for i in range(len(roadlist)-1):
        #Create chartdata (a dataframe of the counts)
        traffic = [df.AmountCars[i], df.AmountTrucks[i], df.AmountBusses[i]]
        chartdata = pd.DataFrame(traffic, index=['Cars', 'Trucks', 'Busses'])
        #Plot the data using 'vincent' so that the plot can appear in a popup
        chart = vincent.Bar(chartdata, width = 200, height = 300)
        trafficpopup = folium.Popup()
        chart.axis_titles(x=(df['LRP Start'][i] + '-' + df['LRP End'][i] + ": " + str("{0:.0f}".format(df.SumTraffic[i])) + " vehicles"), y='')
        folium.Vega(chart, height=350, width=330).add_to(trafficpopup)
        #Create a popup and add it to the traffic marker cluster
        my_PolyLine_Vul=folium.PolyLine(locations=[roadlist[i], roadlist[i+1]],weight= 8, color = df.ColorCode[i], popup = trafficpopup)
        vg.add_child(my_PolyLine_Vul)

    colormap = cm.LinearColormap(['green', 'yellow', 'red'], vmin=0, vmax=100)
    colormap.caption = 'Traffic Density Score'
    mapb.add_child(colormap)
    
    if 'bblist' in globals():
        for i in range(len(bblist)):
            breakbridges = bridges[(bridges.LRPName == bblist[i]) & (bridges.road == 'N1')]
            breakbridges.reset_index(inplace = True, drop = True)
            poplist = [breakbridges.lat[0], breakbridges.lon[0]]
            bridgeinfo = bridgeinfo = '<b>This bridge seems permanently broken!</b><br>Bridge: ' + breakbridges.LRPName[0] + '<br>Type: '  + breakbridges.type[0]
            folium.Marker(poplist, popup = bridgeinfo, icon=folium.Icon(prefix = 'fa', icon = 'road', color='black')).add_to(vg)
    
    if RndBridge == 1:
        breakbridges = bridges[(bridges.LRPName == LRPbridges[z]) & (bridges.road == 'N1')]
        breakbridges.reset_index(inplace = True, drop = True)
        poplist = [breakbridges.lat[0], breakbridges.lon[0]]
        bridgeinfo = bridgeinfo = '<b>This bridge is broken!</b><br>Bridge: ' + breakbridges.LRPName[0] + '<br>Type: '  + breakbridges.type[0]
        folium.Marker(poplist, popup = bridgeinfo, icon=folium.Icon(prefix = 'fa', icon = 'road', color='red')).add_to(vg)
        if z >= 1:
            breakbridges = bridges[(bridges.LRPName == LRPbridges[z-1]) & (bridges.road == 'N1')]
            breakbridges.reset_index(inplace = True, drop = True)
            poplist = [breakbridges.lat[0], breakbridges.lon[0]]
            bridgeinfo = bridgeinfo = '<b>This bridge was broken, but it looks like the problem is fixed now!</b><br>Bridge: ' + breakbridges.LRPName[0] + '<br>Type: '  + breakbridges.type[0]
            folium.Marker(poplist, popup = bridgeinfo, icon=folium.Icon(prefix = 'fa', icon = 'road', color='blue')).add_to(vg)

    legend_html =   '''
                <div style="position: fixed; 
                            bottom: 10px; left: 10px; width: 300px; height: 175px; background-color: white;
                            border:2px solid grey; z-index:9999; font-size:14px;
                            ">&nbsp; <i> Last update at ''' + str((z + 1) % 24) + ''':00 </i> <br> <br>
                              &nbsp; <b> <u> Legend </u> </b> <br>
                              &nbsp; <i class="fa fa-map-marker fa-2x" style="color:black"></i> - Bridge is permanently broken <br>
                              &nbsp; <i class="fa fa-map-marker fa-2x" style="color:red"></i> - Bridge just broke down <br>
                              &nbsp; <i class="fa fa-map-marker fa-2x" style="color:blue"></i> - Bridge has just been repaired <br>
                </div>
                '''

    mapb.get_root().html.add_child(folium.Element(legend_html))
    #Add all the layers to the map
    #Allow fullscreen
    mapb.add_child(pg)
    mapb.add_child(vg)
    mapb.add_child(folium.LayerControl(collapsed = False))
    mapb.add_child(folium.plugins.Fullscreen())
    
    pathlib.Path('PNG_Files').mkdir(parents=True, exist_ok=True) 
    pathlib.Path('HTML_Files').mkdir(parents=True, exist_ok=True) 
    
    mapb.save(outfile = 'HTML_Files/Map_'+RoadName+'_'+str(z)+'.html')
    mapb.save(outfile = 'Map_'+RoadName+'.html')
    #Link to chromedriver
    z = z + 1
    
    
def makegif():
    a = [s for s in os.listdir("PNG_Files/")
         if os.path.isfile(os.path.join("PNG_Files/", s))]
    a.sort(key=lambda s: os.path.getmtime(os.path.join("PNG_Files/", s)))
    print(a)
    with imageio.get_writer('Final_Output_GIF.gif', mode='I') as writer:
        for filename in a:
            image = imageio.imread("PNG_Files/"+filename)
            writer.append_data(image)
    print("You can now rewatch the traffic progress on the N1 road in gif format.")

In [6]:
# exporting data to sql

# Deleting all existing rows and resetting the auto_increment value
def truncate(table):
    cnx = mysql.connector.connect(user='epa1351user',
                                  password='xgt65RR##',
                                  database='epa1351group14')
    cursor = cnx.cursor()
    query = ("TRUNCATE " + table)
    cursor.execute(query)
    cnx.commit()
    cnx.close()

# write a zero to the semaphore function
def writezero():
    cnx = mysql.connector.connect(user='epa1351user',
                              password='xgt65RR##',
                              database='epa1351group14')
    cursor = cnx.cursor()
    query = ("INSERT INTO semaphore (Semaphore) VALUES (0)")
    cursor.execute(query)
    cnx.commit()
    cnx.close()
    
def writezero2():
    truncate("semaphore_randombridge")
    cnx = mysql.connector.connect(user='epa1351user',
                              password='xgt65RR##',
                              database='epa1351group14')
    cursor = cnx.cursor()
    query = ("INSERT INTO semaphore_randombridge (SemaphoreRandomBridge, SemID) VALUES (0, 0)")
    cursor.execute(query)
    cnx.commit()
    cnx.close()
    
def writeone():
    truncate("semaphore_randombridge")
    cnx = mysql.connector.connect(user='epa1351user',
                              password='xgt65RR##',
                              database='epa1351group14')
    cursor = cnx.cursor()
    query = ("INSERT INTO semaphore_randombridge (SemaphoreRandomBridge, SemID) VALUES (1, 0)")
    cursor.execute(query)
    cnx.commit()
    cnx.close()

# function that reads the semaphore table from mysql, which communicates when we can import the traffic data
# returns TRUE if semaphore is 1, FALSE otherwise.
def readsemaphore():
    cnx = mysql.connector.connect(user='epa1351user',
                                  password='xgt65RR##',
                                  database='epa1351group14')
    cursor = cnx.cursor()
    sem = pd.read_sql('SELECT * from semaphore ', con = cnx)
    # need a try/except to catch exceptions when simio truncates the table
    try:
        # semaphore is the only value in this table
        semaphore = sem.iloc[0,0]
        # if this is 1, we want to read the traffic data to python
        if semaphore == 1:
            return True
        # otherwise, we want to wait
        else:
            return False
    except:
        return False
    cnx.commit()
    cnx.close()

# function that takes a string input and writes this to sql

def writebridge(s):
    cnx = mysql.connector.connect(user='epa1351user',
                                  password='xgt65RR##',
                                  database='epa1351group14')
    cursor = cnx.cursor()
    query = ("INSERT INTO writebridge "
            "(bool,bridgeid)" 
            "VALUES (%s,%s)")
    data = (1,s)
    cursor.execute(query,data)
    cnx.commit()
    cnx.close()

def writebridgerandom(s):
    cnx = mysql.connector.connect(user='epa1351user',
                                  password='xgt65RR##',
                                  database='epa1351group14')
    cursor = cnx.cursor()
    query = ("INSERT INTO writebridgerandom "
            "(bool,bridgeid)" 
            "VALUES (%s,%s)")
    data = (1,s)
    cursor.execute(query,data)
    cnx.commit()
    cnx.close()
# Function that asks the user which bridge to break.
# Returns this lrp.

def bridgetobreak():
    validBridge = None
    while validBridge == None:
        global bblist
        bridgeLRP = input("Please enter the LRP of the bridge you intend to break: ")
        # uppercase the input so that lrp001a -> LRP001A
        bridgeLRP = str.upper(bridgeLRP)
        # check that the lrp is indeed connected to a bridge using the list
        if bridgeLRP not in bridgelist:
            print("Sorry, there is no bridge on that LRP")
            continue
        else:
            validBridge = 1
        # lowercase the last letter, so that LRP001A -> LRP001a 
        bridgeLRP = bridgeLRP[:-1] + str.lower(bridgeLRP[-1])
        bblist.append(bridgeLRP)
    return bridgeLRP

def main():
    # delete the value in the semaphore table
    truncate("semaphore")
    # write a zero to the semaphore table
    writezero()
    # delete all values in the table and reset auto increment
    truncate("writebridge")
    truncate('brokenbridge')

    # ask the user how many bridges they want to break. 
    # there is maximum of 3 as this is a limitation from 
    # the simio implementation
    validNumber = None
    while validNumber == None:
        nrBridges = int(input("How many bridges do you want to break? "
                         "Maximum 3 bridges!"))
        if nrBridges == 0 or nrBridges == 1 or nrBridges == 2 or nrBridges == 3:
            validNumber = 1
        else:
            print("Sorry, you can only select 1, 2, or 3 bridges.")
            continue
    # write the assigned number of bridges to SQL
    if nrBridges is not 0:
        global bblist
        bblist = []
        for i in range(nrBridges):
            # define lrp as the input of the user
            lrp = bridgetobreak()
            # ensure this is in the same format as in simio
            simiobridge = 'BridgeN1_' + lrp
            # write this to the sql table
            writebridge(simiobridge)
            i+=1
            
main()

How many bridges do you want to break? Maximum 3 bridges!0


In [7]:
validRnd = None
while validRnd == None:
    RndBridge = input("Aside from your currently chosen bridges, do you want random bridges to break down during your simulation? (Enter 1 or 2) \n1: Yes \n2: No \n")
    RndBridge = int(RndBridge)
    if RndBridge == 1 or RndBridge == 2:
        validRnd = 1
    else:
        print("Sorry, please select a valid answer (1 or 2)")
        continue
shuffle(LRPbridges)
if RndBridge == 1:
    writeone()
elif RndBridge == 2:
    writezero2()
truncate("writebridgerandom")

Aside from your currently chosen bridges, do you want random bridges to break down during your simulation? (Enter 1 or 2) 
1: Yes 
2: No 
2


In [ ]:
validVis = None
while validVis == None:
    Visualisation = input("What is the main focus of your visualisation? (Enter 1 or 2) \n1: Delay Times (recommended) \n2: Traffic Density \n")
    Visualisation = int(Visualisation)
    if Visualisation == 1 or Visualisation == 2:
        validVis = 1
    else:
        print("Sorry, please select a valid answer (1 or 2)")
        continue

What is the main focus of your visualisation? (Enter 1 or 2) 
1: Delay Times (recommended) 
2: Traffic Density 
1


In [ ]:
z = 0
LastEditTime = time.time()
while True:
    #time.sleep(60 - ((time.time() - starttime) % 60))
    if readsemaphore():
        LastEditTime = time.time()
        # read the data and make a map
        if Visualisation == 1:
            makemap1()
        elif Visualisation == 2:
            makemap2()
        truncate("semaphore")
        writezero()
        if RndBridge == 1:
            truncate("writebridgerandom")
            # ensure this is in the same format as in simio
            randomsimiobridge = 'BridgeN1_' + LRPbridges[z]
            # write this to the sql table
            writebridgerandom(randomsimiobridge)
        if z < 2:
        #Generalise pathname so this doesn't need to be adjsuted anymore
            driver = webdriver.Chrome('chromedriver')
            site = os.getcwd()+"/Map_N1.html"
            driver.get(site)
            time.sleep(3)
            driver.save_screenshot('PNG_Files/map_' + str(z) +'.png')
        else:
            driver.refresh()
            time.sleep(3)
            driver.save_screenshot('PNG_Files/map_' + str(z) +'.png')
        print("Frame Loaded. \nIt took "+str((time.time() - LastEditTime))+" seconds for the new frame to load. \n")
    else:
        time.sleep(1)
    #right now just simply stop the python script if the semaphore table hasn't been edited for 10 minutes
    if (z > 144) or ((time.time() - LastEditTime) > 600):
        break
        
makegif()

New update identified! 
Thank you for your patience. Upon completion, the image will be loaded automatically. 
After completion, you can open file: Map_N1_0.html in the HTML_Files folder to inspect the update in further detail.
Frame Loaded. 
It took 29.95317816734314 seconds for the new frame to load. 

New update identified! 
Thank you for your patience. Upon completion, the image will be loaded automatically. 
After completion, you can open file: Map_N1_1.html in the HTML_Files folder to inspect the update in further detail.
Frame Loaded. 
It took 20.234443426132202 seconds for the new frame to load. 

